# EE 538: Low-Noise Analog Circuit Design
## Spring 2021
## Instructor: Jason Silver

## Announcements

 - Assignment 1 due Sunday, April 11 at midnight
    - Jupyter Notebook (.ipyb) submission on Canvas
    - For LTspice problems, include image (e.g. screen capture) of schematic(s)
 - Assignment 2 will be posted Saturday, April 17

## Week 2

 - Motchenbacher Chapters 2 and 3

## Overview

 - Last time...
     - Random variables
     - Thermal noise
     - Noise bandwidth
     - Shot noise 
     - Flicker noise
 - Today
     - 2-port noise theory
     - Noise figure/noise factor
     - Amplifier noise model
     - Correlated noise sources
     - Feedback amplifier noise analysis

## Python packages/modules

In [1]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal
#%matplotlib notebook

mpl.rcParams['font.size'] = 12
mpl.rcParams['legend.fontsize'] = 'large'

def plot_xy(x, y, xlabel, ylabel):
    fig, ax = plt.subplots(figsize=(10.0, 7.5));
    ax.plot(x, y, 'b')
    ax.grid()
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    
def plot_xy2(x1, y1, x1label, y1label, x2, y2, x2label, y2label):
    fig, ax = plt.subplots(2, figsize = (10.0, 7.5));
    ax[0].plot(x1, y1, 'b')
    ax[0].set_ylabel(y1label)
    ax[0].grid()
    
    ax[1].plot(x2, y2, 'b')
    ax[1].set_xlabel(x1label)
    ax[1].set_xlabel(x2label)
    ax[1].set_ylabel(y2label)
    ax[1].grid()
    
    fig.align_ylabels(ax[:])

def plot_xy3(x, y1, y2, y3, xlabel, y1label, y2label, y3label):
    fig, ax = plt.subplots(3, figsize=(10.0,7.5))
    
    ax[0].plot(x, y1)
    ax[0].set_ylabel(y1label)
    ax[0].grid()
    
    ax[1].plot(x, y2)
    ax[1].set_ylabel(y2label)
    ax[1].grid()
    
    ax[2].plot(x, y3)  
    ax[2].set_ylabel(y3label)
    ax[2].set_xlabel(xlabel)
    ax[2].grid()
    
def plot_logxy3(x, y1, y2, y3, xlabel, y1label, y2label, y3label):
    fig, ax = plt.subplots(3, figsize=(10.0,7.5))
    
    ax[0].semilogx(x, y1)
    ax[0].set_ylabel(y1label)
    ax[0].grid()
    
    ax[1].semilogx(x, y2)
    ax[1].set_ylabel(y2label)
    ax[1].grid()
    
    ax[2].semilogx(x, y3)  
    ax[2].set_ylabel(y3label)
    ax[2].set_xlabel(xlabel)
    ax[2].grid()

def plot_logxy(x, y, xlabel, ylabel):
    fig, ax = plt.subplots(figsize=(10.0, 7.5))
    ax.semilogx(x, y, 'b')
    ax.grid();
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    
def plot_loglog(x, y, xlabel, ylabel):
    fig, ax = plt.subplots(figsize=(10.0, 7.5))
    ax.loglog(x, y, 'b')
    ax.grid();
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    
def plot_xlogy(x, y, xlabel, ylabel):
    fig, ax = plt.subplots(figsize=(10.0, 7.5));
    ax.semilogy(x, y, 'b');
    ax.grid();
    ax.set_xlabel(xlabel);
    ax.set_ylabel(ylabel);
    
def read_ltspice_ac(file_name):
    with open(file_name, 'r') as data:
        x = []
        y = []
        z = []
        next(data) # skip header line
        for line in data:
            p = line.split()
            x.append(float(p[0]))
            complex = p[1].split(",")
            y.append(float(complex[0]))
            z.append(float(complex[1]))

    return x, y, z

def plot_logxy2(x1, y1, x2, y2, x1label, y1label, x2label, y2label):
    fig, ax = plt.subplots(2, figsize = (10.0, 7.5));
    ax[0].semilogx(x1, y1, 'b');
    ax[0].set_ylabel(y1label)
    ax[0].grid()
    
    ax[1].semilogx(x2, y2, 'b');
    ax[1].set_xlabel(x1label)
    ax[1].set_xlabel(x2label);
    ax[1].set_ylabel(y2label);
    ax[1].grid();
    
    fig.align_ylabels(ax[:])
    
def plot_noise_bandwidth(f, mag):
    fig, ax = plt.subplots(2, figsize=(10.0,7.5))
    ax[0].semilogx(f, RC_mag)
    ax[0].set_xscale("log")
    ax[0].set_xlim(f[0], f[-1])
    ax[0].set_xticks(np.logspace(0.1,4,5))
    ax[0].set_xticklabels([])
    ax[0].set_ylabel('Magnitude [V/V]')
    ax[0].set_title('Equivalent Noise Bandwidth')
    ax[0].grid()

    ax[1].hlines(1, 0, f_enb, color='tab:blue')
    ax[1].hlines(0, f_enb, f[-1], color='tab:blue')
    ax[1].vlines(f_enb, 0, 1, color='tab:blue')
    ax[1].set_xlim(f[0], f[-1])
    ax[1].set_xscale("log")
    ax[1].set_xticks(np.logspace(0.1,4,5))
    ax[1].set_xticklabels([r'$10^0$',r'$10^1$', r'$10^2$', r'$10^3$', r'$10^4$'])
    ax[1].set_ylabel('Magnitude [V/V]')
    ax[1].set_xlabel('Frequency [Hz]')
    ax[1].grid()
    
def noise_hist(vnoise, vn_rms, bins):
    fig = plt.figure( figsize=(10.0,7.5) )
    vn_norm = vnoise/ vn_rms
    ax = fig.add_subplot(111)
    n, bins, rectangles = ax.hist(vn_norm, bins, density=True, range=(-3, 3),
                                 color='b')
    ax.set_xlabel(r'Sample Voltage [$v_{n(rms)}$]')
    ax.set_ylabel('Probability Density')
    ax.grid()
    fig.canvas.draw()

In [2]:
def fftnoise(f):
    f = np.array(f, dtype='complex')
    Np = (len(f) - 1) // 2
    phases = np.random.rand(Np) * 2 * np.pi
    phases = np.cos(phases) + 1j * np.sin(phases)
    f[1:Np+1] *= phases
    f[-1:-1-Np:-1] = np.conj(f[1:Np+1])
    return np.fft.ifft(f).real

def band_limited_noise(min_freq, max_freq, samples=1024, samplerate=1):
    freqs = np.abs(np.fft.fftfreq(samples, 1/samplerate))
    f = np.zeros(samples)
    idx = np.where(np.logical_and(freqs>=min_freq, freqs<=max_freq))[0]
    f[idx] = 1
    return fftnoise(f)

# Lecture 2 - Noise Modeling

## Noise factor

 - For optimum noise performance, we want to minimize the degradation in the signal-to-noise ratio ($\text{SNR}$) that a system introduces
 
 - The noise factor is defined as
 
\begin{equation}
F \equiv \dfrac{\text{total output noise power}}{\text{output noise due to input source}}
\end{equation}

 - $F$ can also be expressed as the ratio of input $\text{SNR}$ to output $\text{SNR}$. The larger the degradation in $\text{SNR}$, the larger the noise factor
 
 - If a system adds no noise of its own (the ideal case), then the total output noise is due entirely to the source and the noise factor is unity 

## Two-port noise model

<center><img src="img/2port_noise_model.png" width=800 /></center>

 - In the model, all noise appears as inputs to a noiseless network, but arises from the thermal, flicker, and shot noise of the transistors and resistors that make up the network/system
 
 - This noise model allows us to focus on system-level considerations rather than needing to keep track of all the internal noise sources
 
 - The net effect of all internal noise sources can be represented by a single pair of extrenal sources: a noise voltage $\overline{e_n}$ and a noise current $\overline{i_n}$

 - Assuming the noise processes of the source and two-port system are uncorreated (which would be the typical case), the short-circuit mean-square noise current is given by
 
\begin{equation}
\overline{i_{ni}^2} = \overline{i_{ns}^2} + \overline{|i_n + Y_s e_n|^2}
\end{equation}

 - Note that this expression does not assume that $i_n$ and $e_n$ are uncorrelated. In general, $i_n$ contains components that are correlated ($i_c$) and uncorrelated ($i_u$) with $e_n$, such that
 
\begin{equation}
i_n = i_c + i_u
\end{equation}

 - $i_c$ can be related to $e_n$ through a *correlation admittance* $Y_c$, such that $i_c = Y_c e_n$
 
 - Expanding the above expression gives
 
\begin{equation}
\overline{i_{ni}^2} = \overline{i_{ns}^2} + \overline{i_{u}^2} + |Y_s + Y_c|^2\overline{e_n^2}
\end{equation}

 - The *input-referred* noise power can also be expressed in terms of mean-square voltage density
 
\begin{equation}
\overline{e_{ni}^2} = \overline{e_{ns}^2} + \overline{|e_n + Z_s i_n|^2}
\end{equation} 

 - In which case we can define the correlation impedance $Z_c$ as
 
\begin{equation}
Z_c = \dfrac{e_n}{i_c}
\end{equation}

 - Going forward we'll drop the overbars, but we need to remember that we're still dealing with random voltage and current noise sources expresses in terms of $rms$ and mean-square valeus

## Correlation between noise sources

 - Two noise sources are partially correlated when each contains noise that arises from a common phenomenon, in addition to independently generated noise
 
 - As an example, to combine partially correlated voltage noise sources, we can use the general expression
 
\begin{equation}
e_n^2 = e_{n1}^2 + e_{n2}^2 + 2C_c e_{n1} e_{n2}
\end{equation}

 - where $C_c$ is the correlation coefficient
 
 - For completely independent noise sources, $C_c = 0$ and we combine the mean-square quantities directly:
 
\begin{equation}
e_n^2 = e_{n1}^2 + e_{n2}^2
\end{equation}

## Two-port noise factor

 - For the common case where $Y_c = 0$, the input-referred mean-square voltage noise density can be expressed as

\begin{equation}
e_{ni}^2 = e_{ns}^2 + e_n^2 + Z_s^2 i_n^2
\end{equation}

 - The two-port noise factor is thus
 
\begin{equation}
F = \dfrac{e_{ns}^2 + e_n^2 + Z_s^2 i_n^2}{e_{ns}^2} = \boxed{1 + \dfrac{e_n^2 + Z_s^2 i_n^2}{e_{ns}^2}}
\end{equation}

 - The *noise figure* $\text{NF}$ is merely the noise factor in decibels:
 
\begin{equation}
\text{NF} = 10\log{F} = 10\log \left[ 1 + \dfrac{e_n^2 + Z_s^2 i_n^2}{e_{ns}^2} \right]
\end{equation}

## Optimum source resistance

 - To minimize the noise factor/figure, we can take the first derivative of $F$ with respect to the source impedance and set it equal to zero. Assuming $Z_s = R_s$, this yields
 
\begin{equation}
-\dfrac{e_n^2}{R_{opt}^2}+ i_n^2 = 0
\end{equation}

 - From which we find the optimum source resistance to be

\begin{equation}
R_{opt} = \dfrac{e_n}{i_n}
\end{equation}

 - The noise factor corresponding to $R_{opt}$ is thus
 
\begin{equation}
F_{min} = 1 + \dfrac{e_n^2 + R_{opt}^2 i_n^2}{4kT R_{opt}\Delta f} = 1 + \dfrac{2 e_n^2}{4kT \Delta f e_n/i_n} = \boxed{1 + \dfrac{e_n\cdot i_n}{2kT\Delta f}}
\end{equation}


 - Note that there is no direct correspondance between the optimum source resistance and that needed for maximum power transfer 

## Noise resistance and noise temperature

 - The noise resistance of an amplifier is the value of resistance that would generate thermal noise equal to that of the two-port :
 
\begin{equation}
e_n^2 + i_n^2R_s^2 = 4kTR_n \Delta f
\end{equation}

\begin{equation}
R_n = \dfrac{e_n^2+i_n^2R_s^2}{4kT \Delta f}
\end{equation}

## Noise temperature

- Noise temperature is an alternative way of expressing the effect of an amplifier's noise contribution, and is defined as the increase in temperature required for the source resistance to generate the entirety of the amplifier output noise 
 
\begin{equation}
4kT_n R_s \Delta f = e_n^2 + i_n^2 R_s^2
\end{equation}

 - This yields
 
\begin{equation}
T_n = \dfrac{e_n^2 + i_n^2R_s^2}{4kTR_s \Delta f}
\end{equation}

\begin{equation}
F = 1 + \dfrac{T_n}{T_{ref}}
\end{equation}

## Noise in cascaded networks